In [29]:
import time
import os
import sys
from threading import Timer
import random

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from IPython.display import Image
from IPython.display import display

In [28]:
class GameDriver():
    def __init__(self, headless=True):
        options = webdriver.ChromeOptions()
        self.browser = webdriver.Chrome(options=options)
        self.actor = ActionChains(self.browser)

    def navigate(self, url):
        self.browser.get(url) # tells Chrome to go to page
#         try: # waits until load
#             WebDriverWait(self.browser, 3).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))
#         except TimeoutException:
#             print("timeout")
        self.inject_collection_script() # injects event collection script
    
    def step(self, action=None):
        iframe = self.browser.find_element_by_css_selector("iframe#game_drop")
        if action is not None: # before or after?
            self.actor.click(iframe)
            self.actor.send_keys(action).perform()
            
#             order: input action, then delay some, then collect data
            
        events = [] # list of all user events since last step (including recently injected)
#         while True:
#             if self.browser.find_element_by_tag_name("iframe"):
#                 self.browser.switch_to.frame(iframe)
#             self.inject_collection_script()
#             batch = self.browser.execute_script('return window._getRecentEvents();')
#             events.extend(batch)
#             break
        # if events seem suspicious, try to select right iframe or re-inject collection script
        screenshot = iframe.screenshot_as_png # PNG data as bytes
#         screenshot = None
        url = self.browser.current_url # currrent browser location
        metadata = None
        return screenshot, events, url, metadata
    
    def inject_collection_script(self):
        collection_script = '''
        (() => {
        let events = [];
        const inputs = (key, e) => [key, key.startsWith('mouse') ? [e.clientX, e.clientY, e.button] : String.fromCharCode(e.keyCode)]
        ['mousedown', 'mousemove', 'mouseup', 'keyup', 'keydown'].forEach(k => window.addEventListener(k, e => events.push({[+new Date()]: inputs(k,e)}),true));
        window._getRecentEvents = () => events.splice(0, events.length);
        })();
        '''
        self.browser.execute_script(collection_script)
    def close(self):
        self.browser.close()

In [31]:
driver = GameDriver()
driver.navigate("https://mattmakesgames.itch.io/celesteclassic")
screenshots, events, url, metadata = driver.step()
# events, url, metadata = driver.step("c")
# time.sleep(3)
print(events)

# time.sleep(3)
driver.close()

[]


In [33]:
Image(screenshots)
url

'https://mattmakesgames.itch.io/celesteclassic'